## AIGCDetectBenchmark


## Install requirement

In [1]:
!git clone https://github.com/vohoaidanh/ADOF.git

Cloning into 'ADOF'...
remote: Enumerating objects: 850, done.
remote: Counting objects: 100% (364/364), done.
remote: Compressing objects: 100% (205/205), done.
remote: Total 850 (delta 227), reused 261 (delta 143), pack-reused 486 (from 1)
Receiving objects: 100% (850/850), 28.82 MiB | 20.77 MiB/s, done.
Resolving deltas: 100% (521/521), done.


In [2]:
%cd ADOF

/content/ADOF


In [3]:
!pip install -r requirements.txt
!pip install tensorboardX -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 128.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 96.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [4]:
# Without google colab
COLAB = True
if not COLAB:
  !pip install gdown -q
  !apt-get install -y unzip -q
  !apt-get install -y zip -q
  !pip install tensorboardX -q
  !pip install regex -q
  !pip install imageio -q
  !pip install opencv-python -q
  !apt-get install -y libgl1-mesa-glx -q
  !pip install scikit-learn -q
  !pip install scikit-image -q
  !pip install ftfy -q
  !pip install natsort -q
  !pip install blobfile -q

## Download Data

In [5]:
#Download Train, val set
# https://drive.google.com/file/d/1JW-4upuC3Tqq9awz8gnRWEf61NncC8JT/view?usp=drive_link    (Full train, val set)
# https://drive.google.com/file/d/1JLdFyM7JnaUa8y4wBOwNGs3rn4qKH58Z/view?usp=drive_link     (Small tran, val set)
import gdown
DATASET_PATH = 'datasets'
TRAIN_SET_PATH = 'ForenSynths_train'
file_id = '1JLdFyM7JnaUa8y4wBOwNGs3rn4qKH58Z' #Progan train/val 4 class [car, cat, chair, horse]
filename_train = 'progan_train.zip'  # Desired file name and extension
# Construct the download URL
url = f'https://drive.google.com/uc?id={file_id}'
# Download the file
gdown.download(url, filename_train, quiet=False)
#Unzip
!mkdir -p $DATASET_PATH/$TRAIN_SET_PATH
!mkdir -p $DATASET_PATH/$TRAIN_SET_PATH/test
!unzip -q $filename_train -d $DATASET_PATH/$TRAIN_SET_PATH

Downloading...
From (original): https://drive.google.com/uc?id=1JLdFyM7JnaUa8y4wBOwNGs3rn4qKH58Z
From (redirected): https://drive.google.com/uc?id=1JLdFyM7JnaUa8y4wBOwNGs3rn4qKH58Z&confirm=t&uuid=6c860d9b-9fab-4829-b947-97540cd5c9ef
To: /content/ADOF/progan_train.zip
100%|██████████| 2.64G/2.64G [00:26<00:00, 98.2MB/s]


In [6]:
#Download small Test set (this is a small testset just use for reference during training)
import gdown
file_id = '1-02FPvdTaQFDMatEvXAeKLCmnwW6RLx4' #test_set
filename_test = 'test_set.zip'  # Desired file name and extension
# Construct the download URL
url = f'https://drive.google.com/uc?id={file_id}'
# Download the file
gdown.download(url, filename_test, quiet=False)
#Unzip
!unzip -q $filename_test -d $DATASET_PATH/$TRAIN_SET_PATH/test

Downloading...
From (original): https://drive.google.com/uc?id=1-02FPvdTaQFDMatEvXAeKLCmnwW6RLx4
From (redirected): https://drive.google.com/uc?id=1-02FPvdTaQFDMatEvXAeKLCmnwW6RLx4&confirm=t&uuid=1e7da106-ee37-4217-ba48-735669beda46
To: /content/ADOF/test_set.zip
100%|██████████| 1.68G/1.68G [00:18<00:00, 89.2MB/s]

UnZip 6.00 of 20 April 2009, by Debian. Original by Info-ZIP.

Usage: unzip [-Z] [-opts[modifiers]] file[.zip] [list] [-x xlist] [-d exdir]
  Default action is to extract files in list, except those in xlist, to exdir;
  file[.zip] may be a wildcard.  -Z => ZipInfo mode ("unzip -Z" for usage).

  -p  extract files to pipe, no messages     -l  list files (short format)
  -f  freshen existing files, create none    -t  test compressed archive data
  -u  update files, create if necessary      -z  display archive comment only
  -v  list verbosely/show version info       -T  timestamp archive to latest
  -x  exclude files that follow (in xlist)   -d  extract files into exdir
modifiers:
  -n  never overwrite existing files         -q  quiet mode (-qq => quieter)
  -o  overwrite files WITHOUT prompting      -a  auto-convert any text files
  -j  junk paths (do not make directories)   -aa treat ALL files as text
  -U  use escapes for all non-ASCII Unicode  -UU ignore any Unicode fields
  -C  mat

## Train

In [11]:
#experiment-01-no-filter
!find $DATASET_PATH -type d -name "*ipynb*" -exec rm -r {} +
!python train.py \
--name adof-progan-4class- \
--dataroot $DATASET_PATH/$TRAIN_SET_PATH \
--num_thread 2 \
--classes car,cat,chair,horse --batch_size 32 --delr_freq 5 --lr 0.0002 --niter 30

----------------- Options ---------------
                     arch: res50                         
               batch_size: 32                            	[default: 64]
                    beta1: 0.9                           
                blur_prob: 0                             
                 blur_sig: 0.5                           
          checkpoints_dir: ./checkpoints                 
                class_bal: False                         
                  classes: car,cat,chair,horse           	[default: ]
           continue_train: False                         
                 cropSize: 224                           
                 data_aug: False                         
                 dataroot: datasets/ForenSynths_train    	[default: ./dataset/]
                delr_freq: 5                             	[default: 20]
          earlystop_epoch: 15                            
                    epoch: latest                        
              epoch_count:

## Evaluation

In [ ]:
!chmod +x download_dataset.sh
!sed -i 's/\r$//' download_dataset.sh
!./download_dataset.sh

In [18]:
# You should update the test set directory in the "test.py" file.
!find $DATASET_PATH -type d -name "*ipynb*" -exec rm -r {} +
!python test.py \
--model_path ./weights/ADOF_model_epoch_9.pth  \
--num_thread 2 \
--batch_size 32

Model_path ./weights/ADOF_model_epoch_9.pth
                   UniversalFakeDetect
2025_07_12_03_08_40
(0 dalle       ) acc: 90.5; ap: 97.4; r_acc: 0.9; f_acc: 0.9
(1 ldm_100     ) acc: 97.1; ap: 99.8; r_acc: 0.9; f_acc: 1.0
(2 glide_100_10) acc: 97.0; ap: 99.9; r_acc: 0.9; f_acc: 1.0
(3 guided      ) acc: 75.6; ap: 87.4; r_acc: 0.6; f_acc: 0.9
(4 glide_100_27) acc: 97.0; ap: 99.9; r_acc: 0.9; f_acc: 1.0
(5 ldm_200_cfg ) acc: 96.8; ap: 99.8; r_acc: 0.9; f_acc: 1.0
(6 glide_50_27 ) acc: 96.7; ap: 99.8; r_acc: 0.9; f_acc: 1.0
(7 ldm_200     ) acc: 97.0; ap: 99.8; r_acc: 0.9; f_acc: 1.0
(8 Mean      ) acc: 93.5; ap: 98.0
*************************
